In [1]:
import numpy as np
from scipy.integrate import fixed_quad
import plotly.graph_objects as go
import pandas as pd
from iminuit import Minuit
from iminuit.cost import LeastSquares
import matplotlib.pyplot as plt
import os 
import time 

In [2]:
# Load experimental data
atlas_data = pd.read_csv('../../data/ens_atlas_difc0_2.dat', delim_whitespace=True, header=None)
totem_data = pd.read_csv('../../data/ens_totem_difc0_2.dat', delim_whitespace=True, header=None)

# Function to process data for each experiment
def process_data(data, energy_blocks):
    x_values = []
    y_values = []
    y_errors = []
    
    for start, end in energy_blocks:
        block = data.iloc[start:end] if end is not None else data.iloc[start:]
        x_values.append(block[0].values)
        y_values.append(block[1].values)
        y_errors.append(block[2].values)
    
    return x_values, y_values, y_errors

# Energy ranges for each experiment (7TeV, 8TeV, 13TeV)
atlas_blocks = [(0, 29), (29, 58), (58, None)]
totem_blocks = [(0, 65), (65, 118), (118, None)]

# Process data
x_atlas, y_atlas, yerr_atlas = process_data(atlas_data, atlas_blocks)
x_totem, y_totem, yerr_totem = process_data(totem_data, totem_blocks)

# Extract values by energy (index 0=7TeV, 1=8TeV, 2=13TeV)
x_7_atlas, y_7_atlas, yerr_7_atlas = x_atlas[0], y_atlas[0], yerr_atlas[0]
x_8_atlas, y_8_atlas, yerr_8_atlas = x_atlas[1], y_atlas[1], yerr_atlas[1]
x_13_atlas, y_13_atlas, yerr_13_atlas = x_atlas[2], y_atlas[2], yerr_atlas[2]

x_7_totem, y_7_totem, yerr_7_totem = x_totem[0], y_totem[0], yerr_totem[0]
x_8_totem, y_8_totem, yerr_8_totem = x_totem[1], y_totem[1], yerr_totem[1]
x_13_totem, y_13_totem, yerr_13_totem = x_totem[2], y_totem[2], yerr_totem[2]

In [20]:
save_folder = 'run4'

n_points = 10000
b_0 = (33 - 6) / (12 * np.pi)
Lambda = 0.284  # ΛQCD in GeV
gamma_1 = 0.084
gamma_2 = 2.36
rho = 4.0
s0 = 1.0
alpha_prime = 0.25



ensemble_parameters = {
    'atlas': {
        'log': {
            'epsilon': 0.0753,
            'mg': 0.356,
            'a1': 1.373,
            'a2': 2.50
        },
        'pl': {
            'epsilon': 0.0753,
            'mg': 0.421,
            'a1': 1.517,
            'a2': 2.05
        }
    },
    'totem': {
        'log': {
            'epsilon': 0.0892,
            'mg': 0.380,
            'a1': 1.491,
            'a2': 2.77
        },
        'pl':{
            'epsilon': 0.0892,
            'mg': 0.447,
            'a1': 1.689,
            'a2': 1.7
        }
    }
}

ensemble_atlas = 'atlas'  
ensemble_totem = 'totem'

log_model_type = 'log'
pl_model_type = 'pl'   

def get_parameters_with_variations(ensemble_parameters, ensemble_name, model_type, lower_factor=0.7, upper_factor=1.3):
    # Obtém os parâmetros iniciais
    initial_params = ensemble_parameters[ensemble_name][model_type]
    
    # Cria as variações
    initial_params_low = {k: v * lower_factor for k, v in initial_params.items()}
    initial_params_high = {k: v * upper_factor for k, v in initial_params.items()}
    
    return initial_params, initial_params_low, initial_params_high

# Get parameters for selected configuration
initial_params_log_atlas = ensemble_parameters[ensemble_atlas][log_model_type]
initial_params_pl_atlas = ensemble_parameters[ensemble_atlas][pl_model_type]

# Para Atlas
initial_params_log_atlas, initial_params_low_log_atlas, initial_params_high_log_atlas = \
    get_parameters_with_variations(ensemble_parameters, ensemble_atlas, log_model_type)

initial_params_pl_atlas, initial_params_low_pl_atlas, initial_params_high_pl_atlas = \
    get_parameters_with_variations(ensemble_parameters, ensemble_atlas, pl_model_type)

# Para Totem
initial_params_log_totem, initial_params_low_log_totem, initial_params_high_log_totem = \
    get_parameters_with_variations(ensemble_parameters, ensemble_totem, log_model_type)

initial_params_pl_totem, initial_params_low_pl_totem, initial_params_high_pl_totem = \
    get_parameters_with_variations(ensemble_parameters, ensemble_totem, pl_model_type)



In [4]:
def m2_log(q2, mg):
    lambda_squared = Lambda ** 2
    rho_mg_squared = rho * mg ** 2
    ratio = np.log((q2 + rho_mg_squared) / lambda_squared) / np.log(rho_mg_squared / lambda_squared)
    return mg ** 2 * ratio ** (-1 - gamma_1)

def m2_pl(q2, mg):
    lambda_squared = Lambda ** 2
    rho_mg_squared = rho * mg ** 2
    ratio = np.log((q2 + rho_mg_squared) / lambda_squared) / np.log(rho_mg_squared / lambda_squared)
    return (mg ** 4 / (q2 + mg ** 2)) * ratio ** (gamma_2 - 1)


def G_p(q2, a1, a2):
    return np.exp(-(a1 * q2 + a2 * q2 ** 2))

def alpha_D(q2, mg, m2_func):
    m2 = m2_func(q2, mg)
    return 1.0 / (b_0 * (q2 + m2) * np.log((q2 + 4 * m2) / (Lambda ** 2)))

def T_1(k, q, phi, mg, a1, a2, m2_func):
    q2 = q 
    qk_cos = np.sqrt(q) * k * np.cos(phi)
    qk_plus_squared = q2 / 4 + qk_cos + k ** 2
    qk_minus_squared = q2 / 4 - qk_cos + k ** 2
    alpha_D_plus = alpha_D(qk_plus_squared, mg, m2_func)
    alpha_D_minus = alpha_D(qk_minus_squared, mg, m2_func)
    G0 = G_p(q2, a1, a2)
    return alpha_D_plus * alpha_D_minus * G0 ** 2

def T_2(k, q, phi, mg, a1, a2, m2_func):
    q2 = q 
    qk_cos = np.sqrt(q) * k * np.cos(phi)
    qk_plus_squared = q2 / 4 + qk_cos + k ** 2
    qk_minus_squared = q2 / 4 - qk_cos + k ** 2
    alpha_D_plus = alpha_D(qk_plus_squared, mg, m2_func)
    alpha_D_minus = alpha_D(qk_minus_squared, mg, m2_func)
    factor = q2 + 9 * abs(k ** 2 - q2 / 4)
    G0 = G_p(q2, a1, a2)
    G_minus = G_p(factor, a1, a2)
    return alpha_D_plus * alpha_D_minus * G_minus * (2 * G0 - G_minus)

def integrand(y, x, mg, a1, a2, m2_func, q_val, sqrt_s):
    k = sqrt_s * x 
    phi = 2 * np.pi * y
    jacobian = 2 * np.pi * sqrt_s 
    return k * (T_1(k, q_val, phi, mg, a1, a2, m2_func) - T_2(k, q_val, phi, mg, a1, a2, m2_func)) * jacobian 

def sigma_tot(amp_value, s):
    return amp_value.imag / s * 0.389379323

def amp_calculation(diff_T, s, epsilon, t):
    alpha_pomeron = 1.0 + epsilon + alpha_prime * t
    regge_factor = (s**alpha_pomeron) * 1/(s0**(alpha_pomeron-1))
    return 1j * 8 * regge_factor * diff_T  

def differential_sigma(amp_value, s):
    amp_squared = amp_value.imag * amp_value.imag
    denominator = (16 * np.pi * s**2)
    return amp_squared / denominator * 0.389379323


In [5]:
def model_function(x, eps, mg, a1, a2, sqrt_s, model_type='log'):

    # Definindo os parâmetros específicos do modelo
    params = {
        'epsilon': eps,
        'mg': mg,
        'a1': a1,
        'a2': a2
    }
    
    # Escolhendo a massa conforme o modelo
    m2 = m2_log if model_type == 'log' else m2_pl
    
    dif_sigma_lst = []
    
    for q2 in x:
        t = -q2
        
        def inner_integral(x_inner):
            return fixed_quad(
                lambda y: integrand(y, x_inner, params['mg'], params['a1'], 
                                  params['a2'], m2, q2, sqrt_s),
                0, 1,
                n=n_points
            )[0]

        integral_value = fixed_quad(
            inner_integral,
            0, 1,
            n=n_points
        )[0]

        diff_T = integral_value
        s = sqrt_s ** 2
        amp_value = amp_calculation(diff_T, s, params['epsilon'], t)
        dif_sigma_value = differential_sigma(amp_value, s)
        dif_sigma_lst.append(dif_sigma_value)
    
    return np.array(dif_sigma_lst)

In [6]:
def create_least_squares(x, y, yerr, sqrt_s, model_type):
    return LeastSquares(x, y, yerr, 
        lambda x, eps, mg, a1, a2: model_function(x, eps, mg, a1, a2, sqrt_s, model_type))

def total_cost(dataset, model_type):
    return sum([
        create_least_squares(*dataset[7000], 7000, model_type),
        create_least_squares(*dataset[8000], 8000, model_type),
        create_least_squares(*dataset[13000], 13000, model_type)
    ])

# Estrutura dos dados por experimento
atlas_data = {
    7000: (x_7_atlas, y_7_atlas, yerr_7_atlas),
    8000: (x_8_atlas, y_8_atlas, yerr_8_atlas),
    13000: (x_13_atlas, y_13_atlas, yerr_13_atlas)
}

totem_data = {
    7000: (x_7_totem, y_7_totem, yerr_7_totem),
    8000: (x_8_totem, y_8_totem, yerr_8_totem),
    13000: (x_13_totem, y_13_totem, yerr_13_totem)
}

# Custos totais
total_cost_log_atlas = total_cost(atlas_data, 'log')
total_cost_pl_atlas = total_cost(atlas_data, 'pl')
total_cost_log_totem = total_cost(totem_data, 'log')
total_cost_pl_totem = total_cost(totem_data, 'pl')


In [7]:
def otimization(total_cost_func, initial_params, initial_params_low, initial_params_high, model_type: str, ensemble: str):
    print(f"Iniciando otimização dos parâmetros usando LeastSquares para {model_type} em {ensemble.upper()}")

    start_time = time.time()
    
    m = Minuit(total_cost_func, 
               eps=initial_params['epsilon'],
               mg=initial_params['mg'],
               a1=initial_params['a1'],
               a2=initial_params['a2'])
    
    # Configura os limites (±2% dos valores iniciais)
    m.limits['eps'] = (initial_params_low['epsilon'], initial_params_high['epsilon'])
    m.limits['mg'] = (initial_params_low['mg'], initial_params_high['mg'])
    m.limits['a1'] = (initial_params_low['a1'], initial_params_high['a1'])
    m.limits['a2'] = (initial_params_low['a2'], initial_params_high['a2'])
    
    # Configurações adicionais
    # m.strategy = 2
    m.errordef = 1
    m.tol = 0.001
    
    m.migrad()
    # m.simplex()
    m.hesse()
    m.minos(cl=0.9)
    m.migrad()
    
    print(f'Terminado a minimização para {model_type} em {ensemble}')

    execution_time = time.time() - start_time
    
    minutes = int(execution_time // 60)
    seconds = execution_time % 60
    print(f'Tempo de execução para {model_type} em {ensemble}: {minutes} min {seconds:.2f} s')
    
    
    return m




m_log_atlas = otimization(total_cost_log_atlas,
                          initial_params_log_atlas,
                          initial_params_low_log_atlas,
                          initial_params_high_log_atlas, 'log', 'atlas')

m_pl_atlas = otimization(total_cost_pl_atlas,
                         initial_params_pl_atlas,
                         initial_params_low_pl_atlas,
                         initial_params_high_pl_atlas, 'pl', 'atlas')

m_log_totem = otimization(total_cost_log_totem,
                          initial_params_log_totem,
                          initial_params_low_log_totem,
                          initial_params_high_log_totem, 'log', 'totem')

m_pl_totem = otimization(total_cost_pl_totem,
                         initial_params_pl_totem,
                         initial_params_low_pl_totem,
                         initial_params_high_pl_totem, 'pl', 'totem')



Iniciando otimização dos parâmetros usando LeastSquares para log em ATLAS


/home/victor/.local/lib/python3.8/site-packages/iminuit/minuit.py:139: ErrordefAlreadySetWarning: cost function has an errordef attribute equal to 1.0, you should not override this with Minuit.errordef
  warnings.warn(msg, ErrordefAlreadySetWarning)


Terminado a minimização para log em atlas
Tempo de execução para log em atlas: 2 min 45.10 s
Iniciando otimização dos parâmetros usando LeastSquares para pl em ATLAS
Terminado a minimização para pl em atlas
Tempo de execução para pl em atlas: 2 min 15.47 s
Iniciando otimização dos parâmetros usando LeastSquares para log em TOTEM
Terminado a minimização para log em totem
Tempo de execução para log em totem: 7 min 6.44 s
Iniciando otimização dos parâmetros usando LeastSquares para pl em TOTEM
Terminado a minimização para pl em totem
Tempo de execução para pl em totem: 6 min 6.55 s


In [8]:
def get_dif_sigma(epsilon, mg, a1, a2, mg_model):
    
    sqrt_s_values = [7000, 8000, 13000]
    scale_factors = {7000: 1, 8000: 10, 13000: 100}

    sqrt_s = 7000

    def integrand(y, x, mg, a1, a2, m2_func, q_val):
        k = sqrt_s * x 
        phi = 2 * np.pi * y
        jacobian = 2 * np.pi * sqrt_s 
        return k * (T_1(k, q_val, phi, mg, a1, a2, m2_func) - T_2(k, q_val, phi, mg, a1, a2, m2_func)) * jacobian 

    start_q2 = 0.006
    max_q2 = 0.204
    q2_step = 0.001
    n_points = 10000

    results = {}  # Armazena os resultados por energia (√s)
    
    for sqrt_s in sqrt_s_values:
        scale = scale_factors[sqrt_s]
        lst_dif_sigma = []
        lst_q2 = []
        
        q2 = start_q2
        while q2 <= max_q2:
            t = -q2

            def inner_integral(x):
                return fixed_quad(
                    lambda y: integrand(y, x, mg, a1, a2, mg_model, q2),
                    0, 1,
                    n=n_points
                )[0]

            integral_value = fixed_quad(
                inner_integral,
                0, 1,
                n=n_points
            )[0]

            diff_T = integral_value
            s = sqrt_s ** 2
            amp_value = amp_calculation(diff_T, s, epsilon, t)
            dif_sigma_value = differential_sigma(amp_value, s) * scale
            lst_dif_sigma.append(dif_sigma_value)
            lst_q2.append(q2)
            
            q2 += q2_step

        results[sqrt_s] = (lst_q2, lst_dif_sigma)
    
    return results  # Retorna um dicionário com os dados



In [9]:
# for log atlas
dif_sigma_log_atlas = get_dif_sigma(
    m_log_atlas.values['eps'],
    m_log_atlas.values['mg'],
    m_log_atlas.values['a1'],
    m_log_atlas.values['a2'],
    m2_log
)

dif_sigma_log_atlas_7_q2 = dif_sigma_log_atlas[7000][0]
dif_sigma_log_atlas_7_values = dif_sigma_log_atlas[7000][1]

dif_sigma_log_atlas_8_q2 = dif_sigma_log_atlas[8000][0]
dif_sigma_log_atlas_8_values = dif_sigma_log_atlas[8000][1]

dif_sigma_log_atlas_13_q2 = dif_sigma_log_atlas[13000][0]
dif_sigma_log_atlas_13_values = dif_sigma_log_atlas[13000][1]


#for pl atlas
dif_sigma_pl_atlas = get_dif_sigma(
    m_pl_atlas.values['eps'],
    m_pl_atlas.values['mg'], 
    m_pl_atlas.values['a1'],
    m_pl_atlas.values['a2'],
    m2_pl
)

dif_sigma_pl_atlas_7_q2 = dif_sigma_pl_atlas[7000][0]
dif_sigma_pl_atlas_7_values = dif_sigma_pl_atlas[7000][1]

dif_sigma_pl_atlas_8_q2 = dif_sigma_pl_atlas[8000][0]
dif_sigma_pl_atlas_8_values = dif_sigma_pl_atlas[8000][1]

dif_sigma_pl_atlas_13_q2 = dif_sigma_pl_atlas[13000][0]
dif_sigma_pl_atlas_13_values = dif_sigma_pl_atlas[13000][1]


# for log totem
dif_sigma_log_totem = get_dif_sigma(
    m_log_totem.values['eps'],
    m_log_totem.values['mg'],
    m_log_totem.values['a1'],
    m_log_totem.values['a2'],
    m2_log
)

dif_sigma_log_totem_7_q2 = dif_sigma_log_totem[7000][0]
dif_sigma_log_totem_7_values = dif_sigma_log_totem[7000][1]

dif_sigma_log_totem_8_q2 = dif_sigma_log_totem[8000][0]
dif_sigma_log_totem_8_values = dif_sigma_log_totem[8000][1]

dif_sigma_log_totem_13_q2 = dif_sigma_log_totem[13000][0]
dif_sigma_log_totem_13_values = dif_sigma_log_totem[13000][1]


#for pl totem
dif_sigma_pl_totem = get_dif_sigma(
    m_pl_totem.values['eps'],
    m_pl_totem.values['mg'], 
    m_pl_totem.values['a1'],
    m_pl_totem.values['a2'],
    m2_pl
)

dif_sigma_pl_totem_7_q2 = dif_sigma_pl_totem[7000][0]
dif_sigma_pl_totem_7_values = dif_sigma_pl_totem[7000][1]

dif_sigma_pl_totem_8_q2 = dif_sigma_pl_totem[8000][0]
dif_sigma_pl_totem_8_values = dif_sigma_pl_totem[8000][1]

dif_sigma_pl_totem_13_q2 = dif_sigma_pl_totem[13000][0]
dif_sigma_pl_totem_13_values = dif_sigma_pl_totem[13000][1]

In [10]:
#---------------------------------------------------------------- 
# for log atlas
dif_sigma_log_atlas_paper = get_dif_sigma(
    ensemble_parameters['atlas']['log']['epsilon'],
    ensemble_parameters['atlas']['log']['mg'],
    ensemble_parameters['atlas']['log']['a1'],
    ensemble_parameters['atlas']['log']['a2'],
    m2_log
)

dif_sigma_log_atlas_7_q2_paper = dif_sigma_log_atlas_paper[7000][0]
dif_sigma_log_atlas_7_values_paper = dif_sigma_log_atlas_paper[7000][1]

dif_sigma_log_atlas_8_q2_paper = dif_sigma_log_atlas_paper[8000][0]
dif_sigma_log_atlas_8_values_paper = dif_sigma_log_atlas_paper[8000][1]

dif_sigma_log_atlas_13_q2_paper = dif_sigma_log_atlas_paper[13000][0]
dif_sigma_log_atlas_13_values_paper = dif_sigma_log_atlas_paper[13000][1]


#for pl atlas
dif_sigma_pl_atlas_paper = get_dif_sigma(
    ensemble_parameters['atlas']['pl']['epsilon'],
    ensemble_parameters['atlas']['pl']['mg'],
    ensemble_parameters['atlas']['pl']['a1'],
    ensemble_parameters['atlas']['pl']['a2'],
    m2_pl
)

dif_sigma_pl_atlas_7_q2_paper = dif_sigma_pl_atlas_paper[7000][0]
dif_sigma_pl_atlas_7_values_paper = dif_sigma_pl_atlas_paper[7000][1]

dif_sigma_pl_atlas_8_q2_paper = dif_sigma_pl_atlas_paper[8000][0]
dif_sigma_pl_atlas_8_values_paper = dif_sigma_pl_atlas_paper[8000][1]

dif_sigma_pl_atlas_13_q2_paper = dif_sigma_pl_atlas_paper[13000][0]
dif_sigma_pl_atlas_13_values_paper = dif_sigma_pl_atlas_paper[13000][1]


# for log totem
dif_sigma_log_totem_paper = get_dif_sigma(
    ensemble_parameters['totem']['log']['epsilon'],
    ensemble_parameters['totem']['log']['mg'],
    ensemble_parameters['totem']['log']['a1'],
    ensemble_parameters['totem']['log']['a2'],
    m2_log
)

dif_sigma_log_totem_7_q2_paper = dif_sigma_log_totem_paper[7000][0]
dif_sigma_log_totem_7_values_paper = dif_sigma_log_totem_paper[7000][1]

dif_sigma_log_totem_8_q2_paper = dif_sigma_log_totem_paper[8000][0]
dif_sigma_log_totem_8_values_paper = dif_sigma_log_totem_paper[8000][1]

dif_sigma_log_totem_13_q2_paper = dif_sigma_log_totem_paper[13000][0]
dif_sigma_log_totem_13_values_paper = dif_sigma_log_totem_paper[13000][1]


#for pl totem
dif_sigma_pl_totem_paper = get_dif_sigma(
    ensemble_parameters['totem']['pl']['epsilon'],
    ensemble_parameters['totem']['pl']['mg'],
    ensemble_parameters['totem']['pl']['a1'],
    ensemble_parameters['totem']['pl']['a2'],
    m2_pl
)

dif_sigma_pl_totem_7_q2_paper = dif_sigma_pl_totem_paper[7000][0]
dif_sigma_pl_totem_7_values_paper = dif_sigma_pl_totem_paper[7000][1]

dif_sigma_pl_totem_8_q2_paper = dif_sigma_pl_totem_paper[8000][0]
dif_sigma_pl_totem_8_values_paper = dif_sigma_pl_totem_paper[8000][1]

dif_sigma_pl_totem_13_q2_paper = dif_sigma_pl_totem_paper[13000][0]
dif_sigma_pl_totem_13_values_paper = dif_sigma_pl_totem_paper[13000][1]

In [11]:
def add_differential_trace(fig, x, y, label, color='red', mg_model= 'log', legend=True, size = 4, width = 2):
    fig.add_trace(go.Scatter(
        x=x,
        y=y,
        mode='lines+markers',
        line=dict(color=color, width=width),
        marker=dict(size=size),
        name=f'{label}, {mg_model}',
        showlegend=legend,

    ))

def add_data_trace(fig, x, y, y_error, scale=1.0, color='black', size=4,
                           name=None, show_label=True, mode='markers'):
    fig.add_trace(go.Scatter(
        x=x,
        y=y * scale,
        mode=mode,
        marker=dict(color=color, size=size),
        error_y=dict(
            type='data',
            array=y_error * scale,
            visible=True
        ),
        name=name if show_label else None,
        showlegend=show_label
    ))



In [23]:
fig_atlas = go.Figure()

# for log atlas 
add_differential_trace(fig_atlas, dif_sigma_log_atlas_7_q2, dif_sigma_log_atlas_7_values, label='7 TeV', color='red')
add_differential_trace(fig_atlas, dif_sigma_log_atlas_8_q2, dif_sigma_log_atlas_8_values, label ='8 TeV (x10)', color='red')
add_differential_trace(fig_atlas, dif_sigma_log_atlas_13_q2, dif_sigma_log_atlas_13_values, label='13 TeV (x100)', color='red')


# for pl atlas
add_differential_trace(fig_atlas, dif_sigma_pl_atlas_7_q2, dif_sigma_pl_atlas_7_values,label='7 TeV', color='blue', mg_model='pl')
add_differential_trace(fig_atlas, dif_sigma_pl_atlas_8_q2, dif_sigma_pl_atlas_8_values, label='8 TeV (x10)', color='blue' , mg_model='pl')
add_differential_trace(fig_atlas, dif_sigma_pl_atlas_13_q2, dif_sigma_pl_atlas_13_values, label='13 TeV (x100)', color='blue', mg_model='pl')

#-----------------------------------------------------------------------------------------------
add_differential_trace(fig_atlas, dif_sigma_log_atlas_7_q2_paper, 
                       dif_sigma_log_atlas_7_values_paper, label='7 TeV', 
                       color='gray', mg_model='log', legend=False, size = 2, width=1)

add_differential_trace(fig_atlas, dif_sigma_log_atlas_8_q2_paper,
                        dif_sigma_log_atlas_8_values_paper, label='8 TeV (x10)', 
                        color='gray', mg_model='log', legend=False, size = 2, width=1)

add_differential_trace(fig_atlas, dif_sigma_log_atlas_13_q2_paper, 
                       dif_sigma_log_atlas_13_values_paper, label='13 TeV (x100)', 
                       color='gray', mg_model='log', legend=False, size = 2, width=1)



add_differential_trace(fig_atlas, dif_sigma_pl_atlas_7_q2_paper, 
                       dif_sigma_pl_atlas_7_values_paper, label='7 TeV', 
                       color='gray', mg_model='pl', legend=False, size = 2, width=1)

add_differential_trace(fig_atlas, dif_sigma_pl_atlas_8_q2_paper, 
                       dif_sigma_pl_atlas_8_values_paper, label='8 TeV (x10)', 
                       color='gray', mg_model='pl', legend=False, size = 2, width=1)

add_differential_trace(fig_atlas, dif_sigma_pl_atlas_13_q2_paper, 
                       dif_sigma_pl_atlas_13_values_paper, label='13 TeV (x100)', 
                       color='gray', mg_model='pl', legend=False, size = 2, width=1)
#-----------------------------------------------------------------------------------------------

#data points
add_data_trace(fig_atlas, x_7_atlas, y_7_atlas, yerr_7_atlas, name='ATLAS 7 TeV', show_label=True, mode='markers')
add_data_trace(fig_atlas, x_8_atlas, y_8_atlas*10, yerr_8_atlas, name='ATLAS 8 TeV', show_label=True, mode='markers')
add_data_trace(fig_atlas, x_13_atlas, y_13_atlas*100, yerr_13_atlas, name='ATLAS 13 TeV', show_label=True, mode='markers')

# Atualiza layout
fig_atlas.update_layout(
    title='dσ/dt vs. |t| - Log and PL models in ATLAS',
    xaxis_title='|t| (GeV²)',
    yaxis_title='dσ/dt (mb/GeV²)',
    yaxis_type='log',
    legend_title='Mass Model',
    plot_bgcolor='white',
    hovermode='x unified'
)

fig_atlas.update_xaxes(gridcolor='lightgray')
fig_atlas.update_yaxes(gridcolor='lightgray')

os.makedirs(f'../../results/dif_sigma/{save_folder}', exist_ok=True)

# fig_atlas.show(renderer='browser')
fig_atlas.write_html(f'../../results/dif_sigma/{save_folder}/atlas_differential_cross_section_minimized.html')
fig_atlas.write_image(f'../../results/dif_sigma/{save_folder}/atlas_differential_cross_section_minimized.pdf', width=1200, height=600)

In [22]:
fig_totem = go.Figure()

# for log totem 
add_differential_trace(fig_totem, dif_sigma_log_totem_7_q2, dif_sigma_log_totem_7_values, label='7 TeV', color='red')
add_differential_trace(fig_totem, dif_sigma_log_totem_8_q2, dif_sigma_log_totem_8_values, label='8 TeV (x10)', color='red')
add_differential_trace(fig_totem, dif_sigma_log_totem_13_q2, dif_sigma_log_totem_13_values, label='13 TeV (x100)', color='red')

# for pl totem
add_differential_trace(fig_totem, dif_sigma_pl_totem_7_q2, dif_sigma_pl_totem_7_values, label='7 TeV', color='blue', mg_model='pl')
add_differential_trace(fig_totem, dif_sigma_pl_totem_8_q2, dif_sigma_pl_totem_8_values, label='8 TeV (x10)', color='blue', mg_model='pl')
add_differential_trace(fig_totem, dif_sigma_pl_totem_13_q2, dif_sigma_pl_totem_13_values, label='13 TeV (x100)', color='blue', mg_model='pl')

#-----------------------------------------------------------------------------------------------
add_differential_trace(fig_totem, dif_sigma_log_totem_7_q2_paper, 
                       dif_sigma_log_totem_7_values_paper, label='7 TeV', 
                       color='gray', mg_model='log', legend=False, size = 2, width=1)

add_differential_trace(fig_totem, dif_sigma_log_totem_8_q2_paper, 
                       dif_sigma_log_totem_8_values_paper, label='8 TeV (x10)', 
                       color='gray', mg_model='log', legend=False, size = 2, width=1)

add_differential_trace(fig_totem, dif_sigma_log_totem_13_q2_paper, 
                       dif_sigma_log_totem_13_values_paper, label='13 TeV (x100)', 
                       color='gray', mg_model='log', legend=False, size = 2, width=1)



add_differential_trace(fig_totem, dif_sigma_pl_totem_7_q2_paper, 
                       dif_sigma_pl_totem_7_values_paper, label='7 TeV', 
                       color='gray', mg_model='pl', legend=False, size = 2, width=1)

add_differential_trace(fig_totem, dif_sigma_pl_totem_8_q2_paper, 
                       dif_sigma_pl_totem_8_values_paper, label='8 TeV (x10)', 
                       color='gray', mg_model='pl', legend=False, size = 2, width=1)

add_differential_trace(fig_totem, dif_sigma_pl_totem_13_q2_paper, 
                       dif_sigma_pl_totem_13_values_paper, label='13 TeV (x100)', 
                       color='gray', mg_model='pl', legend=False, size = 2, width=1)

#-----------------------------------------------------------------------------------------------

# data points
add_data_trace(fig_totem, x_7_totem, y_7_totem, yerr_7_totem, name='TOTEM 7 TeV', show_label=True, mode='markers')
add_data_trace(fig_totem, x_8_totem, y_8_totem * 10, yerr_8_totem, name='TOTEM 8 TeV', show_label=True, mode='markers')
add_data_trace(fig_totem, x_13_totem, y_13_totem * 100, yerr_13_totem, name='TOTEM 13 TeV', show_label=True, mode='markers')


# Atualiza layout
fig_totem.update_layout(
    title='dσ/dt vs. |t| - Log and PL models in TOTEM',
    xaxis_title='|t| (GeV²)',
    yaxis_title='dσ/dt (mb/GeV²)',
    yaxis_type='log',
    legend_title='Mass Model',
    plot_bgcolor='white',
    hovermode='x unified'
)


fig_totem.update_xaxes(gridcolor='lightgray')
fig_totem.update_yaxes(gridcolor='lightgray')

os.makedirs(f'../../results/dif_sigma/{save_folder}', exist_ok=True)

# fig_totem.show(renderer='browser')
fig_totem.write_html(f'../../results/dif_sigma/{save_folder}/totem_differential_cross_section_minimized.html')
fig_totem.write_image(f'../../results/dif_sigma/{save_folder}/totem_differential_cross_section_minimized.pdf', width=1200, height=600)


In [14]:
data_sigma_tot_atlas = pd.read_csv(
    "../../data/sigma_tot_2/ensemble_StRh_atlas.dat",
    delim_whitespace=True,
    header=None,
    nrows=70
)

data_sigma_tot_totem = pd.read_csv(
    "../../data/sigma_tot_2/ensemble_StRh_totem.dat",  # Supondo que existe um arquivo similar para TOTEM
    delim_whitespace=True,
    header=None,
    nrows=80
)

x_sigma_tot_atlas = data_sigma_tot_atlas[0].to_numpy()
y_sigma_tot_atlas = data_sigma_tot_atlas[1].to_numpy()
y_error_sigma_tot_atlas = data_sigma_tot_atlas[2].to_numpy()

x_sigma_tot_totem = data_sigma_tot_totem[0].to_numpy()
y_sigma_tot_totem = data_sigma_tot_totem[1].to_numpy()
y_error_sigma_tot_totem = data_sigma_tot_totem[2].to_numpy()

start_sqrt_s = 1
max_sqrt_s = 13010
step = 100

def add_total_trace(fig, x, y, color='red', label='', line_style='solid', legend=True, size = 3, width = 2):
    fig.add_trace(go.Scatter(
        x=x,
        y=y,
        mode='lines+markers',
        line=dict(color=color, width=width, dash=line_style),
        marker=dict(size=size),
        name = label, 
        showlegend=legend
    ))

def get_sigma_tot(epsilon, mg, a1, a2, mg_model):

    lst_sigma_tot = []
    lst_sqrt_s = []

    sqrt_s = start_sqrt_s

    while sqrt_s <= max_sqrt_s:
        def integrand(y, x, mg, a1, a2, m2_func):
            k = sqrt_s * x
            phi = 2 * np.pi * y
            jacobian = 2 * np.pi * sqrt_s

            return k * (T_1(k, 0.0, phi, mg, a1, a2, m2_func) - T_2(k, 0.0, phi, mg, a1, a2, m2_func)) * jacobian
        
        def amp_calculation(diff_T, s, epsilon):
            alpha_pomeron = 1.0 + epsilon
            regge_factor = (s / s0) ** alpha_pomeron
            
            return 1j * 8.0 * regge_factor * diff_T

        s = sqrt_s ** 2

        integral_value = fixed_quad(
            lambda x: fixed_quad(
                lambda y: integrand(y, x, mg, a1, a2, mg_model),
                0, 1, n=n_points)[0],

            0, 1, n=n_points)[0]
        
        lst_sigma_tot.append(sigma_tot(
            amp_calculation(integral_value, s, epsilon), s))
        
        lst_sqrt_s.append(sqrt_s)
        sqrt_s += step
    return lst_sigma_tot, lst_sqrt_s


In [21]:
#-----------------------------------------------------------------------------------------------
sigma_tot_log_atlas_paper = get_sigma_tot(
    ensemble_parameters['atlas']['log']['epsilon'],
    ensemble_parameters['atlas']['log']['mg'],
    ensemble_parameters['atlas']['log']['a1'],
    ensemble_parameters['atlas']['log']['a2'],
    m2_log
)

sigma_tot_pl_atlas_paper = get_sigma_tot(
    ensemble_parameters['atlas']['pl']['epsilon'],
    ensemble_parameters['atlas']['pl']['mg'],
    ensemble_parameters['atlas']['pl']['a1'],
    ensemble_parameters['atlas']['pl']['a2'],
    m2_pl
)
sigma_tot_log_totem_paper = get_sigma_tot(
    ensemble_parameters['totem']['log']['epsilon'],
    ensemble_parameters['totem']['log']['mg'],
    ensemble_parameters['totem']['log']['a1'],
    ensemble_parameters['totem']['log']['a2'],
    m2_log
)

sigma_tot_pl_totem_paper = get_sigma_tot(
    ensemble_parameters['totem']['pl']['epsilon'],
    ensemble_parameters['totem']['pl']['mg'],
    ensemble_parameters['totem']['pl']['a1'],
    ensemble_parameters['totem']['pl']['a2'],
    m2_pl
)


lst_sqrt_s = sigma_tot_log_atlas_paper[1]

sigma_tot_log_atlas_values_paper = sigma_tot_log_atlas_paper[0]
sigma_tot_pl_atlas_values_paper = sigma_tot_pl_atlas_paper[0]
sigma_tot_log_totem_values_paper = sigma_tot_log_totem_paper[0]
sigma_tot_pl_totem_values_paper = sigma_tot_pl_totem_paper[0]

#-----------------------------------------------------------------------------------------------

sigma_tot_log_atlas = get_sigma_tot(
    m_log_atlas.values['eps'],
    m_log_atlas.values['mg'],
    m_log_atlas.values['a1'],
    m_log_atlas.values['a2'],
    m2_log
)

sigma_tot_pl_atlas = get_sigma_tot(
    m_pl_atlas.values['eps'],
    m_pl_atlas.values['mg'],
    m_pl_atlas.values['a1'],
    m_pl_atlas.values['a2'],
    m2_pl
)

sigma_tot_log_totem = get_sigma_tot(
    m_log_totem.values['eps'],
    m_log_totem.values['mg'],
    m_log_totem.values['a1'],
    m_log_totem.values['a2'],
    m2_log
)

sigma_tot_pl_totem = get_sigma_tot(
    m_pl_totem.values['eps'],
    m_pl_totem.values['mg'],
    m_pl_totem.values['a1'],
    m_pl_totem.values['a2'],
    m2_pl
)

lst_sqrt_s = sigma_tot_log_atlas[1]


sigma_tot_log_atlas_values = sigma_tot_log_atlas[0]
sigma_tot_pl_atlas_values = sigma_tot_pl_atlas[0]

sigma_tot_log_totem_values = sigma_tot_log_totem[0]
sigma_tot_pl_totem_values = sigma_tot_pl_totem[0]

fig = go.Figure()

add_total_trace(fig, lst_sqrt_s, sigma_tot_log_atlas_values, color='red', label='Log Atlas', line_style='solid')
add_total_trace(fig, lst_sqrt_s, sigma_tot_pl_atlas_values, color='blue', label='PL Atlas', line_style='solid')

add_total_trace(fig, lst_sqrt_s, sigma_tot_log_totem_values, color='red', label='Log Totem', line_style='dot')
add_total_trace(fig, lst_sqrt_s, sigma_tot_pl_totem_values, color='blue', label='PL Totem', line_style='dot')

#-----------------------------------------------------------------------------------------------
add_total_trace(fig, lst_sqrt_s, sigma_tot_log_atlas_values_paper, color='gray', label='Log Atlas (Paper)', line_style='solid', legend=False, size = 2, width=1)
add_total_trace(fig, lst_sqrt_s, sigma_tot_pl_atlas_values_paper, color='gray', label='PL Atlas (Paper)', line_style='solid', legend=False, size = 2, width=1)
add_total_trace(fig, lst_sqrt_s, sigma_tot_log_totem_values_paper, color='gray', label='Log Totem (Paper)', line_style='solid', legend=False, size = 2, width=1)
add_total_trace(fig, lst_sqrt_s, sigma_tot_pl_totem_values_paper, color='gray', label='PL Totem (Paper)', line_style='solid', legend=False, size = 2, width=1)

#----

add_data_trace(fig, x_sigma_tot_atlas, y_sigma_tot_atlas, y_error_sigma_tot_atlas, name='ATLAS', show_label=True, mode='markers')
add_data_trace(fig, x_sigma_tot_totem, y_sigma_tot_totem, y_error_sigma_tot_totem, name='TOTEM', show_label=True, mode='markers')

fig.update_layout(
    title = 'σ_tot vs. √s - Ensemble Atlas and Totem in Log and PL model',
    xaxis=dict(
        title='√s [GeV]',
        type='log',
        range=[np.log10(2000), np.log10(14000)],
    ),
    yaxis=dict(
        title='σ_tot [mb]',
        range=[80, 125]
    ),
    showlegend=True,
    legend=dict(
        title='Ensembles'
    ),
    plot_bgcolor='white',
    hovermode='x unified'
)
    
fig.update_xaxes(gridcolor='lightgray')
fig.update_yaxes(gridcolor='lightgray')


os.makedirs(f'../../results/sigma_tot/{save_folder}', exist_ok=True)

# fig.show(renderer="browser")
fig.write_html(f'../../results/sigma_tot/{save_folder}/sigma_tot_models_minimized.html')
fig.write_image(f'../../results/sigma_tot/{save_folder}/sigma_tot_models_minimized_zoom.pdf', width=1200, height=600)